In [ ]:
import pandas as pd

file1 = 'TimeSeriesForecasting/input/features.csv'
file2 = 'TimeSeriesForecasting/input/train.csv'

full_table1 = pd.read_csv(file1)
full_table2 = pd.read_csv(file2)

print(full_table1.columns)
print(full_table2.columns)